In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy.stats import spearmanr
from scipy.stats import norm

import statsmodels.api as sm

Histograms of the annual maxima peak; determination of the new start of the block maxima according to the dry season

New files of the annual maxima sets (starting during the dry season)

In [ ]:
conf = pd.read_csv(r"C:\Users\Fonse_ma\OneDrive - Stichting Deltares\Desktop\Maria\Notebooks\HBV\Confluences_ok2.csv", delimiter=';')

In [ ]:
c = 'C68'
op = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HIDnew\01_TSonefile\TS_" + c + ".csv" 
TS_C68 = pd.read_csv(op, delimiter=';')

TS_C68 = TS_C68[~(TS_C68['Q_S1'] < 0)]
TS_C68 = TS_C68[~(TS_C68['Q_S2'] < 0)]
TS_C68 = TS_C68[~(TS_C68['Q_S2'] == 999)]

In [ ]:
#From RQ1
c = 'C68'
op = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HIDnew\03_AMhyd" + r"\disc1d_" + c
AM1_C68 = pd.read_pickle(op + "_AM1.pkl")
AM2_C68 = pd.read_pickle(op + "_AM2.pkl")
AM3_C68 = pd.read_pickle(op + "_AM3.pkl")

In [ ]:
#With the new start of the block maxima -> Sep 
c = 68
op = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HIDnew\05_HYDzoom" + r"\C%d_" %c + r"disc1d_"
    
AM1 = pd.read_pickle(op + "AM1.pkl")
AM2 = pd.read_pickle(op + "AM2.pkl")
AM3 = pd.read_pickle(op + "AM3.pkl")

### Histogram

In [ ]:
AM1_hist = []
AM2_hist = []
AM3_hist = []

for i in range(1, 13):
    AM1_hist.append(len(AM1_C68[AM1_C68.month == i])) 
    AM2_hist.append(len(AM2_C68[AM2_C68.month == i])) 
    AM3_hist.append(len(AM3_C68[AM3_C68.month == i])) 

In [ ]:
plt.figure(figsize=(8, 4))
x = np.linspace(1, 12, 12)
xlabels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
plt.bar(x, AM1_hist, label='Set 1: S1max-S2conc', color='Blue', edgecolor= 'k', width=0.9, zorder=3)

plt.title(f'Histogram C68', fontsize=18) #runoff
plt.xticks(ticks=x ,labels=xlabels, fontsize=12, rotation=90)
plt.ylabel(f'Number of peaks', fontsize=16) #Q m3/s
plt.yticks(fontsize=12)
#plt.xlim(left=0, right=1)
# plt.ylim(ymin = 0, ymax = ytext+1000)
plt.legend(fontsize=12, bbox_to_anchor=(1,1), loc="upper left")
plt.grid(color='lightgray', zorder=-1)
plt.tight_layout()

plt.figure(figsize=(8, 4))
x = np.linspace(1, 12, 12)
xlabels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
plt.bar(x, AM2_hist, label='Set 2: S2max-S1conc', color='Blue', edgecolor= 'k', width=0.9, zorder=3)

plt.title(f'Histogram C68', fontsize=18) #runoff
plt.xticks(ticks=x ,labels=xlabels, fontsize=12, rotation=90)
plt.ylabel(f'Number of peaks', fontsize=16) #Q m3/s
plt.yticks(fontsize=12)
#plt.xlim(left=0, right=1)
# plt.ylim(ymin = 0, ymax = ytext+1000)
plt.legend(fontsize=12, bbox_to_anchor=(1,1), loc="upper left")
plt.grid(color='lightgray', zorder=-1)
plt.tight_layout()

plt.figure(figsize=(8, 4))
x = np.linspace(1, 12, 12)
xlabels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
plt.bar(x, AM3_hist, label='Set 3: Cmax (C=S1+S2)', color='Blue', edgecolor= 'k', width=0.9, zorder=3)

plt.title(f'Histogram C68', fontsize=18) #runoff
plt.xticks(ticks=x ,labels=xlabels, fontsize=12, rotation=90)
plt.ylabel(f'Number of peaks', fontsize=16) #Q m3/s
plt.yticks(fontsize=12)
#plt.xlim(left=0, right=1)
# plt.ylim(ymin = 0, ymax = ytext+1000)
plt.legend(fontsize=12, bbox_to_anchor=(1,1), loc="upper left")
plt.grid(color='lightgray', zorder=-1)
plt.tight_layout()

The month where the least of the peaks occur is September.

So lets change the start of the year for the Annual Maxima selection

Functions from HYD02-03_AM

In [ ]:
def AddSavg(am1, S1_savg, S2_savg, C_savg):
    conditionsALL = [am1.season=='DJF', am1.season=='JJA', am1.season=='MAM', am1.season=='SON']

    choicesS1 = [S1_savg['DJF'], S1_savg['JJA'], S1_savg['MAM'], S1_savg['SON']]
    choicesS2 = [S2_savg['DJF'], S2_savg['JJA'], S2_savg['MAM'], S2_savg['SON']]
    choicesC = [C_savg['DJF'], C_savg['JJA'], C_savg['MAM'], C_savg['SON']]

    am1['S1savg'] = np.select(conditionsALL, choicesS1, default='ERROR').astype(float)
    am1['S2savg'] = np.select(conditionsALL, choicesS2, default='ERROR').astype(float)
    am1['Csavg'] = np.select(conditionsALL, choicesC, default='ERROR').astype(float)
    am1['S1/S1savg'] = am1.Q_S1 / am1.S1savg
    am1['S2/S2savg'] = am1.Q_S2 / am1.S2savg
    am1['C/Csavg'] = am1.Q_C / am1.Csavg
    return am1

def AMnew(conf_):
    """
    Creates...
    ds: dictionary with all the Time series per station 
    conf: configuration of the confluences
    c: confluence to be analyzed
    w: window of the multi-day events
    Returns: ds_
    """
    
    #Delete the rows with values lower than 0, or equal to 999
    conf_1 = conf_[~(conf_['Q_S1'] < 0)]
    conf_1 = conf_1[~(conf_1['Q_S2'] < 0)]
    conf_1 = conf_1[~(conf_1['Q_S2'] == 999)]
    conf_1    
    
    #Seasonal averages
    conditions = [
        (conf_1.month==1)|(conf_1.month==2)|(conf_1.month==12),
        (conf_1.month==3)|(conf_1.month==4)|(conf_1.month==5),
        (conf_1.month==6)|(conf_1.month==7)|(conf_1.month==8),
        (conf_1.month==9)|(conf_1.month==10)|(conf_1.month==11)]

    choices = ['DJF', 'MAM', 'JJA', 'SON']
    conf_1['season'] = np.select(conditions, choices, default='ERROR')
    
    savg_S1 = conf_1.groupby(['season'])['Q_S1'].mean()
    savg_S2 = conf_1.groupby(['season'])['Q_S2'].mean()
    savg_C = conf_1.groupby(['season'])['Q_C'].mean()

    
    #2 Extreme selection sets
    AM1_id = conf_1.groupby(['yearAM'])['Q_S1'].idxmax()
    AM2_id = conf_1.groupby(['yearAM'])['Q_S2'].idxmax()
    AM3_id = conf_1.groupby(['yearAM'])['Q_C'].idxmax()

    am1 = conf_1.loc[AM1_id[:]]
    am2 = conf_1.loc[AM2_id[:]]
    am3 = conf_1.loc[AM3_id[:]]
    
    return am1, am2, am3, savg_S1, savg_S2, savg_C

In [ ]:
# To create the new files..   already fone and oppened avobe

In [ ]:
am1, am2, am3, savg_S1, savg_S2, savg_C = AMnew(TS_C68)

AM1 = AddSavg(am1, savg_S1, savg_S2, savg_C)
AM2 = AddSavg(am2, savg_S1, savg_S2, savg_C)
AM3 = AddSavg(am3, savg_S1, savg_S2, savg_C)

In [ ]:
c=68
sv = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HIDnew\05_HYDzoom" + r"\C%d_" %c + r"disc1d_"
    
AM1.to_pickle(sv + "AM1.pkl")
AM2.to_pickle(sv + "AM2.pkl")
AM3.to_pickle(sv + "AM3.pkl")

In [ ]:
AM1_hist2 = []
AM2_hist2 = []
AM3_hist2 = []

for i in range(1, 13):
    AM1_hist2.append(len(AM1[AM1.month == i])) 
    AM2_hist2.append(len(AM2[AM2.month == i])) 
    AM3_hist2.append(len(AM3[AM3.month == i])) 
    
plt.figure(figsize=(10, 4))
x = np.linspace(1, 12, 12)
xlabels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
plt.bar(x-0.2, AM1_hist2, label='Set 1: S1max-S2conc', color='Blue', edgecolor= 'k', width=0.2, zorder=3)
plt.bar(x, AM2_hist2, label='Set 2: S2max-S1conc', color='Green', edgecolor= 'k', width=0.2, zorder=3)
plt.bar(x+0.2, AM3_hist2, label='Set 3: Cmax (C=S1+S2)', color='Orange', edgecolor= 'k', width=0.2, zorder=3)

plt.title(f'Histogram C68', fontsize=18) #runoff
plt.xticks(ticks=x ,labels=xlabels, fontsize=12, rotation=90)
plt.ylabel(f'Number of peaks', fontsize=16) #Q m3/s
plt.yticks(fontsize=12)
#plt.xlim(left=0, right=1)
# plt.ylim(ymin = 0, ymax = ytext+1000)
plt.legend(fontsize=12, bbox_to_anchor=(1,1), loc="upper left")
plt.grid(color='lightgray', zorder=-1)
plt.tight_layout()

plt.figure(figsize=(8, 4))
x = np.linspace(1, 12, 12)
xlabels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
plt.bar(x, AM1_hist2, label='Set 1: S1max-S2conc', color='Blue', edgecolor= 'k', width=0.9, zorder=3)

plt.title(f'Histogram C68', fontsize=18) #runoff
plt.xticks(ticks=x ,labels=xlabels, fontsize=12, rotation=90)
plt.ylabel(f'Number of peaks', fontsize=16) #Q m3/s
plt.yticks(fontsize=12)
#plt.xlim(left=0, right=1)
# plt.ylim(ymin = 0, ymax = ytext+1000)
plt.legend(fontsize=12, bbox_to_anchor=(1,1), loc="upper left")
plt.grid(color='lightgray', zorder=-1)
plt.tight_layout()

plt.figure(figsize=(8, 4))
x = np.linspace(1, 12, 12)
xlabels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
plt.bar(x, AM2_hist2, label='Set 2: S2max-S1conc', color='Blue', edgecolor= 'k', width=0.9, zorder=3)

plt.title(f'Histogram C68', fontsize=18) #runoff
plt.xticks(ticks=x ,labels=xlabels, fontsize=12, rotation=90)
plt.ylabel(f'Number of peaks', fontsize=16) #Q m3/s
plt.yticks(fontsize=12)
#plt.xlim(left=0, right=1)
# plt.ylim(ymin = 0, ymax = ytext+1000)
plt.legend(fontsize=12, bbox_to_anchor=(1,1), loc="upper left")
plt.grid(color='lightgray', zorder=-1)
plt.tight_layout()

plt.figure(figsize=(8, 4))
x = np.linspace(1, 12, 12)
xlabels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
plt.bar(x, AM3_hist2, label='Set 3: Cmax (C=S1+S2)', color='Blue', edgecolor= 'k', width=0.9, zorder=3)

plt.title(f'Histogram C68', fontsize=18) #runoff
plt.xticks(ticks=x ,labels=xlabels, fontsize=12, rotation=90)
plt.ylabel(f'Number of peaks', fontsize=16) #Q m3/s
plt.yticks(fontsize=12)
#plt.xlim(left=0, right=1)
# plt.ylim(ymin = 0, ymax = ytext+1000)
plt.legend(fontsize=12, bbox_to_anchor=(1,1), loc="upper left")
plt.grid(color='lightgray', zorder=-1)
plt.tight_layout()

In [ ]:
def plotSsets(AM1, AM2, AM3, c, td, unit, peaks, color):
#     AM1 = pd.read_pickle(op + "AM1.pkl")
#     AM2 = pd.read_pickle(op + "AM2.pkl")
#     AM3 = pd.read_pickle(op + "AM3.pkl")
    
    AM1['ind'] = np.arange(len(AM1))
    AM2['ind'] = np.arange(len(AM2))
    AM3['ind'] = np.arange(len(AM3))
    randomindex1 = np.random.choice(AM1.ind, size=peaks, replace=False)
    set1 = pd.DataFrame()
    set2 = pd.DataFrame()
    set3 = pd.DataFrame()
    for i in range(peaks):
        set1 = set1.append(AM1.loc[AM1.ind == randomindex1[i]])
        set2 = set2.append(AM2.loc[AM2.ind == randomindex1[i]])
        set3 = set3.append(AM3.loc[AM3.ind == randomindex1[i]])
    
    
    plt.figure(figsize=(7, 7))
    plt.plot(set1['Q_S1'], set1['Q_S2'], 'o', color='none', markeredgecolor=color,markersize=10, label='Set 1: S1max-S2conc')
    plt.plot(set2['Q_S1'], set2['Q_S2'], 'x', color=color, markersize=9, label='Set 2: S2max-S1conc')
    plt.plot(set3['Q_S1'], set3['Q_S2'], '.', color=color, markersize=10, label='Set 3: Cmax (C=S1+S2)')
    plt.title(f'{td} | C{c+1} ({peaks} peaks AM)', fontsize=20, y=1.05)
    plt.xlabel(f'S1 ({unit})', fontsize=18)
    plt.ylabel(f'S2 ({unit})', fontsize=18)
    plt.xlim(left=0, right=12000)
    plt.ylim(ymin = 0, ymax =4500)
    plt.xticks(fontsize=12, rotation=90)
    plt.yticks(fontsize=12)
    plt.legend(fontsize=18, bbox_to_anchor=(1,1), loc="upper left")

    plt.grid()
    return

In [ ]:
c = 67
peaks = 50
color = 'Blue'
unit = 'm3/s'
td = 'HYD | Discharge 1d'
plotSsets(AM1, AM2, AM3, c, td, unit, peaks, color)

### Correlation

In [ ]:
#Using the natural year
rho1, Sp1 = spearmanr(AM1_C68['Q_S1'], AM1_C68['Q_S2'])
rho2, Sp2 = spearmanr(AM2_C68['Q_S1'], AM2_C68['Q_S2'])
rho3, Sp3 = spearmanr(AM3_C68['Q_S1'], AM3_C68['Q_S2'])
print(f'AM1_rho={rho1:.4}, AM2_rho={rho2:.4}, AM3_rho={rho3:.4}')

In [ ]:
#Using September as the start of the year
rho1, Sp1 = spearmanr(AM1['Q_S1'], AM1['Q_S2'])
rho2, Sp2 = spearmanr(AM2['Q_S1'], AM2['Q_S2'])
rho3, Sp3 = spearmanr(AM3['Q_S1'], AM3['Q_S2'])
print(f'AM1_rho={rho1:.4}, AM2_rho={rho2:.4}, AM3_rho={rho3:.4}')